## Create COLVAR with PLUMED

In [1]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'

In [6]:
from ase.io import read
import numpy as np

Load data

In [3]:
folder = './'
traj_path = folder+'traj-std-ev10.xyz'

traj = read(traj_path,index=':')
len(traj)

3001

Create PLUMED input file

In [7]:
from ase.data import covalent_radii as CR
from ase.data import chemical_symbols as CS
from itertools import combinations

def write_plumed_coordination_input(atoms,save_path='plumed.dat',colvar_path='COLVAR'):
    """ Write plumed input file to calculate coordination numbers between species
        The parameters for the switching functions are calculated from covalent radii
    """ 

    with open(save_path, 'w') as f:
        # Get symbols
        symbols = list(set(atoms.get_chemical_symbols()))
        symbols = sorted(symbols,key=lambda s: CS.index(s),reverse=True)

        print('UNITS LENGTH=A',file=f)

        for symbol in symbols:
            idxs = np.argwhere ( [ s == symbol for s in atoms.get_chemical_symbols()] )[:,0]
            print(f'{symbol}: GROUP ATOMS={",".join( [str(i+1) for i in  idxs ])}',file=f )

        for s1,s2 in combinations(symbols,2):
            r_0 = (CR[CS.index(s1)]+CR[CS.index(s2)]) * 1.5
            print(f'coord_{s1}-{s2}: COORDINATION GROUPA={s1} GROUPB={s2} R_0={r_0:.2f} NN=6 MM=12', file=f)

        for symbol in symbols:
            idxs = np.argwhere ( [ s == symbol for s in atoms.get_chemical_symbols()] )[:,0]
            if len(idxs) == 2:
                print(f'dist_{symbol}-{symbol}: DISTANCE ATOMS={idxs[0]+1},{idxs[1]+1}', file=f)                                                                                                                                                                

        print(f'PRINT ARG=* STRIDE=1 FILE={colvar_path}',file=f)

In [8]:
plumed_path =f'{folder}/plumed.dat'
colvar_path=f'{folder}/COLVAR'
write_plumed_coordination_input(atoms=traj[0],
                                save_path=plumed_path,
                                colvar_path=colvar_path)

!cat {plumed_path}

UNITS LENGTH=A
Co: GROUP ATOMS=2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144
Fe: GROUP ATOMS=1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,99,101,103,105,107,109,111,113,115,117,119,121,123,125,127,129,131,133,135,137,139,141,143
N: GROUP ATOMS=145,146
coord_Co-Fe: COORDINATION GROUPA=Co GROUPB=Fe R_0=3.87 NN=6 MM=12
coord_Co-N: COORDINATION GROUPA=Co GROUPB=N R_0=2.96 NN=6 MM=12
coord_Fe-N: COORDINATION GROUPA=Fe GROUPB=N R_0=3.05 NN=6 MM=12
dist_N-N: DISTANCE ATOMS=145,146
PRINT ARG=* STRIDE=1 FILE=.//COLVAR


Run PLUMED driver

In [11]:
!plumed driver --plumed {plumed_path} --ixyz {traj_path} --box {','.join( str(i) for i in traj[0].get_cell().flatten() ) } --length-units A

plumed driver --plumed .//plumed.dat --ixyz ./traj-std-ev10.xyz --box 11.372,0.0,0.0,0.0,12.061827473480129,0.0,0.0,0.0,30.051522894566773 --length-units A
